In [3]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel



/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


2023-06-02 08:47:07.921548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 08:47:08.481516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home6/svstaalduine/.local/lib/python3.9/site-packages/cv2/../../lib64:/sw/arch/Centos8/EB_production/2021/software/ZeroMQ/4.3.4-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/util-linux/2.36-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/libsodium/1.0.18-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/OpenPGM/5.2.122-GCCcore-10.3.0/lib:/sw

In [4]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230118_Plate537/"
upload_targ = "/DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate537/Analysis/"

In [5]:
test_name = glob(directory_targ + '*')[0]
imgs_address = test_name
print(test_name)

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230118_Plate537/Fluorescence_ex_20220118_Plate537.xlsx


In [6]:

plate_list = glob(directory_targ + "*_Plate*_*")
datadict = {'address_total' : plate_list}
dataframe = pd.DataFrame(data=datadict)
dataframe['video_name'] = [address.split('/')[-1] for address in dataframe['address_total']]
dataframe['plate_nr'] = [address.split('/')[-1].split('_')[-2][5:] for address in dataframe['address_total']]
dataframe['video_nr'] = [int(address.split('/')[-1].split('_')[-1]) for address in dataframe['address_total']]
dataframe['date_imaged'] = [address.split('/')[-1][:8] for address in dataframe['address_total']]

parent_folder = []
data_table = []
magnification = []
fps = []
mode = []

for row in dataframe.iloc:
#     print(row)
    parent_folder.append(str(Path(row['address_total']).parent))
    excel_file = glob(str(Path(row['address_total']).parent) + f'/*{row["date_imaged"]}*{row["plate_nr"]}.xl*')
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
#         print(row['video_name'])
#         print(excel_table['Unnamed: 0'][21])
#         print(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'])
        magnification.append(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'].iloc[0])
        fps.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['FPS'].iloc[0])
        mode.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Bright-field (BF)\nor\nFluorescence (F)'].iloc[0])
        
    else:
        csv_file = glob(str(Path(row['address_total']).parent) + f'/{row["date_imaged"]}*{row["plate_nr"]}.csv')
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
            df_comma = pd.read_csv(csv_file[0], nrows=1,sep=",")
            df_semi = pd.read_csv(csv_file[0], nrows=1, sep=";")
            if df_comma.shape[1]>df_semi.shape[1]:
                csv_table = pd.read_csv(csv_file[0], sep=",")
            else:
                csv_table = pd.read_csv(csv_file[0], sep=";")
            magnification.append(csv_table[csv_table['video']==int(row['video_name'].split("_")[-1])]['Lens'].iloc[0])
            fps.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['fps'].iloc[0])
            mode.append(csv_table.loc[csv_table['video'] == int(row['video_name'].split("_")[-1])]['Illumination'].iloc[0])

        else:
            print("Halp! No datatable found! Pls add the excel file to the folders.")

dataframe['fps'] = fps
dataframe['magnification'] = magnification
dataframe['mode'] = mode
dataframe['parent_folder'] = parent_folder
dataframe['data_table'] = data_table

dataframe = dataframe.sort_values(by='video_name')
# dataframe = dataframe[dataframe['video_nr']== 5]
# dataframe = dataframe[dataframe['date_imaged'] == '20221026']
# print(len(dataframe))


if dataframe.isnull().values.any():
    print("Found NaNs in the excel files! Blame the experimentalists.")
    dataframe = dataframe.interpolate(method='pad', limit_direction='forward')
if dataframe.isnull().values.any():
    raise("This excel sheet is unworkable, please ask the responsible person")
dataframe

,address_total,video_name,plate_nr,video_nr,date_imaged,fps,magnification,mode,parent_folder,data_table
79,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_001,537,1,20220118,10.0,4.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
26,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_002,537,2,20220118,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
80,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_003,537,3,20220118,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
56,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_004,537,4,20220118,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
23,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_005,537,5,20220118,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
...,...,...,...,...,...,...,...,...,...,...
5,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_085,537,85,20220118,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
53,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_086,537,86,20220118,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
31,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_087,537,87,20220118,10.0,4.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
8,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20220118_Plate537_088,537,88,20220118,20.0,50.0,F,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...


In [7]:
# FINAL CHECK FOR SOURCE FOLDER AND UPLOAD FOLDER

# Please make sure that the upload folder is correct, 
# as the program WILL overwrite that which is already there.
print(directory_targ)
print(upload_targ)

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230118_Plate537/
/DATA/FLUORESCENCE/DATA_NileRed/20230118_Plate537/Analysis/


In [8]:
nr_parallel = np.min([len(dataframe.index), 16])

run_parallel(
    "flux_extract.py",
    [directory_targ, 15, 0.95, 0.001, 60, upload_targ],
    dataframe,
    nr_parallel,
    "1:00:00",
    "flux_extract"
)

/home/svstaalduine/bash/job.sh
Sending jobs with id 1685688463934565040
89
Submitted batch job 2834445


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2834446
Submitted batch job 2834447


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2834448


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2834449
